In [1]:
import os
import numpy as np
import cv2
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
directory = "TImages/"

In [3]:
image_size = (244, 244)
classes = ["0", "1", "2"]

In [4]:
# Create an ImageDataGenerator instance with data augmentation settings
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    samplewise_center=True,
    samplewise_std_normalization=True,
)

In [5]:
training_data = []

def create_training_data():
    for category in classes:
        path = os.path.join(directory, category)
        class_num = classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, image_size)

                # Generate and store augmented images
                augmented_images = []
                augmented_images.append(new_array)  # Original image
                img_array_aug = new_array.reshape((1,) + new_array.shape)
                i = 0
                for batch in datagen.flow(img_array_aug, batch_size=1):
                    augmented_images.append(batch[0])
                    i += 1
                    if i >= 5:  # Generate 5 augmented images per input image
                        break

                for augmented_image in augmented_images:
                    image_hsv = cv2.cvtColor(augmented_image, cv2.COLOR_BGR2HSV)
                    training_data.append([image_hsv, class_num])

            except Exception as e:
                pass

In [6]:
create_training_data()

In [7]:
lenofimage = len(training_data)
print(lenofimage)

5850


In [8]:
# Shuffle the training data to ensure randomness
np.random.shuffle(training_data)

In [9]:
# Separate images (X) and labels (y)
X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)

# Convert lists to numpy arrays
X = np.array(X)
y = np.array(y)

In [10]:
# Print the shapes to verify
print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

Shape of X: (5850, 244, 244, 3)
Shape of y: (5850,)


In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [14]:
# Define the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(244, 244, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(classes), activation='softmax'))

In [15]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [16]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
147/147 [==============================] - 213s 1s/step - loss: 7.3335 - accuracy: 0.9120 - val_loss: 7.0055 - val_accuracy: 0.9359
Epoch 2/10
147/147 [==============================] - 240s 2s/step - loss: 2.0265 - accuracy: 0.9216 - val_loss: 11.2212 - val_accuracy: 0.9333
Epoch 3/10
147/147 [==============================] - 237s 2s/step - loss: 1.4398 - accuracy: 0.9278 - val_loss: 4.1366 - val_accuracy: 0.9333
Epoch 4/10
147/147 [==============================] - 239s 2s/step - loss: 0.6530 - accuracy: 0.9301 - val_loss: 4.4337 - val_accuracy: 0.9359
Epoch 5/10
147/147 [==============================] - 242s 2s/step - loss: 1.6158 - accuracy: 0.9293 - val_loss: 7.1729 - val_accuracy: 0.9350
Epoch 6/10
147/147 [==============================] - 243s 2s/step - loss: 5.1964 - accuracy: 0.9239 - val_loss: 4.6590 - val_accuracy: 0.9325
Epoch 7/10
147/147 [==============================] - 211s 1s/step - loss: 1.2076 - accuracy: 0.9335 - val_loss: 3.5922 - val_accuracy: 0.934

In [17]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 242, 242, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 121, 121, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 119, 119, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 59, 59, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 57, 57, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 28, 28, 128)       0

In [21]:
from tensorflow.keras.models import Model

# Create a new model that extracts features from the last convolutional layer
feature_extraction_model = Model(inputs=model.input, outputs=model.get_layer('flatten').output)

# Extract features from your dataset
X_train_features = feature_extraction_model.predict(X_train)
X_test_features = feature_extraction_model.predict(X_test)


37/37 [==============================] - 8s 201ms/step


In [20]:
# from sklearn.svm import SVC

# # Create and train an SVM classifier
# svm_classifier = SVC(kernel='linear', C=1.0, verbose=2)
# svm_classifier.fit(X_train_features, y_train)

# # Evaluate the SVM classifier on the validation set
# svm_accuracy = svm_classifier.score(X_test_features, y_test)
# print("SVM Validation Accuracy:", svm_accuracy)


In [22]:
## for NAIVE BAYES

In [23]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [24]:
# Feature Scaling
sc = StandardScaler()

X_train_nb = sc.fit_transform(X_train_features)
X_test_nb = sc.transform(X_test_features)

In [25]:
# Define the parameter grid for hyperparameter tuning
param_grid = {
    'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5] # You can adjust this range
    
}

In [26]:
# Create the Gaussian Naive Bayes model
nb = GaussianNB()

In [27]:
# Create the GridSearch object without cross-validation
grid_search = GridSearchCV(nb, param_grid, cv=5, verbose=2)  # Set cv=None for no cross-validation

In [28]:
# priors
grid_search.class_prior_ = [0.5, 0.5]

In [30]:
# Fit the GridSearch object to your data
grid_search.fit(X_train_nb, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] END ................................var_smoothing=1e-09; total time=   5.8s
[CV] END ................................var_smoothing=1e-09; total time=   5.8s
[CV] END ................................var_smoothing=1e-09; total time=   5.9s
[CV] END ................................var_smoothing=1e-09; total time=   5.8s
[CV] END ................................var_smoothing=1e-09; total time=   5.7s
[CV] END ................................var_smoothing=1e-08; total time=   5.7s
[CV] END ................................var_smoothing=1e-08; total time=   5.8s
[CV] END ................................var_smoothing=1e-08; total time=   5.8s
[CV] END ................................var_smoothing=1e-08; total time=   5.9s
[CV] END ................................var_smoothing=1e-08; total time=   5.9s
[CV] END ................................var_smoothing=1e-07; total time=   5.9s
[CV] END ................................var_smoo

GridSearchCV(cv=5, estimator=GaussianNB(),
             param_grid={'var_smoothing': [1e-09, 1e-08, 1e-07, 1e-06, 1e-05]},
             verbose=2)

In [33]:
# Get the best estimator and evaluate it
best_nb = grid_search.best_estimator_
y_test_pred_nb = best_nb.predict(X_test_nb)
accuracy_nb = accuracy_score(y_test, y_test_pred_nb)
print("Best Gaussian Naive Bayes Parameters:", grid_search.best_params_)
print("Accuracy: {:.2f}%".format(accuracy_nb * 100))

Best Gaussian Naive Bayes Parameters: {'var_smoothing': 1e-09}
Accuracy: 77.69%
